In [1164]:
import numpy as np

In [1165]:
class ANN:
    w_l_ij= None
    b_l_j = None
    a_l_ij = None
    delta_l_ij = None
    input_featureCount = None
    input_count = None
    layer_info = None
    dl_dw = None
    def __init__(self,xShape,rseed = 10,listLayer= None):
        np.random.seed(rseed)
        inputCount,inputDim = xShape
        self.input_featureCount = inputDim
        self.input_count = inputCount
        self.layer_info = [['None',inputDim]]
        self.w_l_ij = ['None']
        self.b_l_j = ['None']
        self.optimizer = None

    def compile(self,optimizer,lossFunction):
        self.optimizer = optimizer
        self.layer_info.append(['Loss',lossFunction])

    def __str__(self):
        return

    def addLayers_Dense(self,neuronCount,activationFunction):
        if(len(self.layer_info) == 1) :
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.input_featureCount,neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)
        else:
            self.layer_info.append(['Dense',activationFunction,neuronCount])
            w = np.random.rand(self.layer_info[-2][2],neuronCount)
            b = np.random.rand(1,neuronCount)
            self.w_l_ij.append(w)
            self.b_l_j.append(b)

    def forward_layer_dense(self,layerNo):
        if layerNo < 1:
            print("ERRRROR")
        output = self.layer_info[layerNo][1](self.a_l_ij[layerNo-1] @ self.w_l_ij[layerNo] + self.b_l_j[layerNo]) 
        self.a_l_ij.append(output)

    def forward_layer_loss(self,layerNo,y):
        y_hat  =  self.a_l_ij[-1]
        lossFunction = self.layer_info[layerNo][1]
        return lossFunction(y_hat,y)

    def forwardProp(self,x,y):
        self.a_l_ij = []
        self.a_l_ij.append(x)
        for layerNo in range(len(self.layer_info)):
            if self.layer_info[layerNo][0] == 'Dense':
                self.forward_layer_dense(layerNo)
            if self.layer_info[layerNo][0] == 'Loss':
                cost = self.forward_layer_loss(layerNo,y)
                return cost
            
    def backwardProp(self,y):
        self.dl_dw = []
        lastIndex = len(self.layer_info) - 2
        self.delta_l_ij = [i for i in range(0,lastIndex+1)]
        self.delta_l_ij[lastIndex] = (2 * (self.a_l_ij[lastIndex] - y).T * self.diff_activation_linear(self.a_l_ij[lastIndex]))
        for layerNo in range(lastIndex,0,-1):
            self.dl_dw.append(self.a_l_ij[layerNo-1].T @ self.delta_l_ij[layerNo])
            self.delta_l_ij[layerNo-1] = (self.w_l_ij[layerNo] @self.delta_l_ij[layerNo].T).T * self.diff_activation_linear(self.a_l_ij[layerNo-1])
        self.dl_dw.append(['None'])
        self.dl_dw.reverse()

    def fit(self,x,y):
        #Hyperparameters if required: 
        epoch = 100
        eta = 1e-7
        self.optimizer(self,x,y,eta,epoch)

    @staticmethod    
    def optimizer_gradientDescent(Obj,x,y,eta,epoch):
        for i in  range(epoch):
            errorSum = 0
            print("Epoch:",i,end="\t")
            for j in range(y.shape[1]):
                #x and y both are 2d matrix
                x_1xd = x[j:j+1]
                y_1xk = y[j:j+1]
                errorSum += Obj.forwardProp(x_1xd,y_1xk)
                Obj.backwardProp(y_1xk)
                for index in range(1,len(Obj.w_l_ij)):
                    Obj.w_l_ij[index] = Obj.w_l_ij[index] - (eta * Obj.dl_dw[index])
                    Obj.b_l_j[index] = Obj.b_l_j[index] - (eta * Obj.delta_l_ij[index])
            print("Error:",errorSum)

    @staticmethod
    def loss_MSE(P,y):
        return np.sum((P - y)**2)   
      
    @staticmethod
    def activation_linear(z):
        return z

    def diff_activation_linear(self,x):
        return np.ones(x.shape)
    
    @staticmethod
    def activation_sigmoid(z):    
        return 1 / (1 + np.exp(-z))
    
    @staticmethod
    def activation_tanh(z):
        pass

    @staticmethod
    def activation_relu(z):
        pass

    @staticmethod
    def activation_softmax(z):
        pass



    


In [1166]:
import pandas as pd
import numpy as np
import sklearn.datasets as sk

In [1167]:
dictt = sk.fetch_california_housing()
x = dictt.data
y = np.array([dictt.target])
y = y.T
x[0:1].shape

(1, 8)

In [1168]:
model = ANN(x.shape,20)

In [1169]:
model.addLayers_Dense(3,ANN.activation_linear)
model.addLayers_Dense(4,ANN.activation_linear)
model.addLayers_Dense(1,ANN.activation_linear)

In [1170]:
model.compile(ANN.optimizer_gradientDescent,ANN.loss_MSE)

In [1171]:
model.layer_info

[['None', 8],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 3],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 4],
 ['Dense', <function __main__.ANN.activation_linear(z)>, 1],
 ['Loss', <function __main__.ANN.loss_MSE(P, y)>]]

In [1172]:
model.delta_l_ij

In [1173]:
model.b_l_j

['None',
 array([[0.77025193, 0.49238104, 0.63125307]]),
 array([[0.31051155, 0.62685344, 0.55744981, 0.31857956]]),
 array([[0.59813382]])]

In [1174]:
model.dl_dw

In [1175]:
model.a_l_ij

In [1176]:
import copy
w_old = copy.deepcopy(model.w_l_ij)

In [1177]:
model.fit(x,y)

Epoch: 0	Error: 206715.4986939546
Epoch: 1	Error: 151012.47285261957
Epoch: 2	Error: 115067.68124891495
Epoch: 3	Error: 90295.46451928669
Epoch: 4	Error: 72405.67441240819
Epoch: 5	Error: 59026.448655801294
Epoch: 6	Error: 48745.93879763876
Epoch: 7	Error: 40674.72858684923
Epoch: 8	Error: 34226.690173462
Epoch: 9	Error: 29001.088413045025
Epoch: 10	Error: 24715.408115443628
Epoch: 11	Error: 21165.233038541606
Epoch: 12	Error: 18199.31619207668
Epoch: 13	Error: 15703.559157746302
Epoch: 14	Error: 13590.413156565226
Epoch: 15	Error: 11791.686894450842
Epoch: 16	Error: 10253.555756973336
Epoch: 17	Error: 8933.02894593063
Epoch: 18	Error: 7795.403600026705
Epoch: 19	Error: 6812.400320539465
Epoch: 20	Error: 5960.777533553459
Epoch: 21	Error: 5221.2877874210135
Epoch: 22	Error: 4577.881830266371
Epoch: 23	Error: 4017.094671113685
Epoch: 24	Error: 3527.566970206857
Epoch: 25	Error: 3099.668230812958
Epoch: 26	Error: 2725.197398205843
Epoch: 27	Error: 2397.142912022143
Epoch: 28	Error: 2109.

In [1178]:
w_new = model.w_l_ij

In [1179]:
w_old,w_new

(['None',
  array([[0.5881308 , 0.89771373, 0.89153073],
         [0.81583748, 0.03588959, 0.69175758],
         [0.37868094, 0.51851095, 0.65795147],
         [0.19385022, 0.2723164 , 0.71860593],
         [0.78300361, 0.85032764, 0.77524489],
         [0.03666431, 0.11669374, 0.7512807 ],
         [0.23921822, 0.25480601, 0.85762553],
         [0.94977903, 0.56168686, 0.17878052]]),
  array([[0.83949792, 0.4610394 , 0.49794007, 0.67941112],
         [0.65078591, 0.26879524, 0.06732467, 0.77144514],
         [0.48098413, 0.32920641, 0.51064106, 0.26362883]]),
  array([[0.39484322],
         [0.25797459],
         [0.58224112],
         [0.16162871]])],
 ['None',
  array([[0.58434739, 0.89632925, 0.88857854],
         [0.7972049 , 0.02907131, 0.67721864],
         [0.37550698, 0.51734949, 0.65547484],
         [0.19338494, 0.27214614, 0.71824288],
         [0.63666975, 0.79677923, 0.66106099],
         [0.03550293, 0.11626875, 0.75037448],
         [0.22200354, 0.24850659, 0.84419296],